# Module 02: Plan and implement Azure Cosmos DB SQL API

- [[Learning path]](https://docs.microsoft.com/en-us/learn/paths/plan-implement-azure-cosmos-db-sql-api/?ns-enrollment-type=Collection&ns-enrollment-id=1k8wcz8zooj2nx)
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/02-configure-throughput.html): Configure throughput for Azure Cosmos DB SQL API with the Azure portal
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/03-migrate-data.html): Migrate existing data using Azure Data Factory

# Plan Resource Requirements
## Understand throughput
### Container-level throughput provisioning

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/plan-resource-requirements/media/2-container.png)

### Database-level throughput provisioning

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/plan-resource-requirements/media/2-database.png)

### Mixed-throughput provisioning

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/plan-resource-requirements/media/2-mixed.png)

## Evaluate throughput requirements

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/plan-resource-requirements/media/3-request-units.png)

| **Operation type** | **Number of requests per second** | **Number of RU per request** | **RU/s needed** |
| ---: | :---: | :---: | :--- |
| **Write Single Document** | 10,000 | 10 | 100,000 |
| **Top Query #1** | 700 | 100 | 70,000 |
| **Top Query #2** | 200 | 100 | 20,000 |
| **Top Query #3** | 100 | 100 | 10,000 |
| **Total RU/s** | | | 200,000 RU/s |

## Evaluate data storage requirements

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/plan-resource-requirements/media/4-calculator.png)

## Time-to-live (TTL)

The TTL value for a container is configured using the ``DefaultTimeToLive`` property of the container's JSON object.

| **DefaultTimeToLive** | **Expiration** |
| --- | --- |
| *Does not exist* | Items are not automatically expired |
| ``-1`` | Items will not expire by default |
| *n* | *n* seconds after last modified time |

Example:

| **Container.DefaultTimeToLive** | **Item.ttl** | **Expiration in seconds** |
| :--- | :--- | ---: |
| ``1000`` | *null* | ``1000`` |
| ``1000`` | ``-1`` |  *This item will never expire* |
| ``1000`` | ``2000`` | ``2000`` |

Another example: 

| **Container.DefaultTimeToLive** | **Item.ttl** | **Expiration in seconds** |
| :--- | :--- | ---: |
| *null* | *null* | *This item will never expire* |
| *null* | ``-1`` |  *This item will never expire* |
| *null*| ``2000`` | *This item will never expire* |

## Plan for data retention with time-to-live (TTL)

Consider solutions such to aggregate and migrate data such as:
- Change feed
- Azure Data Warehouse
- Azure Blob Storage

# Configure Azure Cosmos DB SQL API throughput

## Serverless

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-azure-cosmos-db-sql-api/media/2-serverless.png)

## Compare serverless vs. provisioned throughput

| **Throughput**  | **Workloads** | **RUs** | **Global Distribution** | **Compare storage limits** |
|---|---|---|---|---|
| **Provisioned** | Ideal for workloads with predictable traffic patterns | Number RUs per second preset to each container | Can distribute data to an unlimited number of Azure regions | Unlimited data in a container |
| **Serverless**  | Can handle workloads that have wildly varying traffic | Doesn't require any planning or automatic provisioning | Accounts can only run in a single Azure region | Up to 50 GB of data in a container |

## Autoscale

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/configure-azure-cosmos-db-sql-api/media/4-autoscale-2.png)

## Compare autoscale vs. standard (manual) throughput

| **Scaling**  | **Workloads** | **RUs** | **Scenarios** | **Rate-limiting** |
|---|---|---|---|---|
| **Standard** | Suited for workloads with steady traffic | Requires a static number of request units to be assigned ahead of time | Where the application throughput can be accurately predicted | Since the RU/s are static, requests beyond this will be rate-limited |
| **Autoscale**  | Suited for unpredictable traffic | You only set the maximum RUs | Where the application throughput can't be accurately predicted, but an acceptable max throughput can be assigned | Will scale up to the max RU/s before similarly rate-limiting responses |


# Moving data into and out of Azure Cosmos DB SQL API

## Move data by using Azure Data Factory

### Linked service

```json
{
    "name": "<example-name-of-linked-service>",
    "properties": {
        "type": "CosmosDb",
        "typeProperties": {
            "connectionString": "AccountEndpoint=<cosmos-endpoint>;AccountKey=<cosmos-key>;Database=<cosmos-database>"
        }
    }
}
```

![image](https://docs.microsoft.com/en-us/azure/data-factory/media/connector-azure-cosmos-db/azure-cosmos-db-connector.png)
![image]()https://docs.microsoft.com/en-us/azure/data-factory/media/connector-azure-cosmos-db/configure-azure-cosmos-db-linked-service.png

### Read from Azure Cosmos DB

```json
{
    "source": {
        "type": "CosmosDbSqlApiSource",
        "query": "SELECT id, categoryId, price, quantity, name FROM products WHERE price > 500",
        "preferredRegions": [
            "East US",
            "West US"
        ]        
    }
}
```

### Write to Azure Cosmos DB

```json
"sink": {
    "type": "CosmosDbSqlApiSink",
    "writeBehavior": "upsert"
}
```

## Move data by using a Kafka connector

### Write to Azure Cosmos DB

```bash
# Create a new topic named prodlistener
kafka-topics --create \
    --zookeeper localhost:2181 \
    --topic prodlistener \
    --replication-factor 1 \
    --partitions 1

# Start producer
kafka-console-producer \
    --broker-list localhost:9092 \
    --topic hotels
```

### Read from Azure Cosmos DB

```json
{
  "name": "cosmosdb-source-connector",
  "config": {
    "connector.class": "com.azure.cosmos.kafka.connect.source.CosmosDBSourceConnector",
    "tasks.max": "1",
    "key.converter": "org.apache.kafka.connect.json.JsonConverter",
    "value.converter": "org.apache.kafka.connect.json.JsonConverter",
    "connect.cosmos.task.poll.interval": "100",
    "connect.cosmos.connection.endpoint": "<cosmos-endpoint>",
    "connect.cosmos.master.key": "<cosmos-key>",
    "connect.cosmos.databasename": "<cosmos-database>",
    "connect.cosmos.containers.topicmap": "<kafka-topic>#<cosmos-container>",
    "connect.cosmos.offset.useLatest": false,
    "value.converter.schemas.enable": "false",
    "key.converter.schemas.enable": "false"
  }
}
```

## Move data by using Stream Analytics

![image](https://docs.microsoft.com/en-us/azure/architecture/reference-architectures/data/images/stream-processing-asa/stream-processing-asa.png)

| **Property** | **Description** |
| ---: | :--- |
| ``Output alias`` | An alias to refer to this output in the query |
| ``Account ID`` | Account endpoint URI |
| ``Account Key`` | Account key |
| ``Database`` | Name of the database resource |
| ``Container name`` | Name of the container |

## Move data by using the Azure Cosmos DB Spark connector

![image](https://docs.microsoft.com/en-us/azure/cosmos-db/media/synapse-link/synapse-analytics-cosmos-db-architecture.png)

```bash
# Setup the cosmos db with synapse link enabled (--enable-analytical-storage true and --analytical-storage-ttl -1)
az cosmosdb create --name <name> --resource-group <resource-group> --enable-analytical-storage true
az cosmosdb sql container create --resource-group <resource-group> --account <account> --database <database> --name <name> --partition-key-path <partition-key-path> --throughput <throughput> --analytical-storage-ttl -1
```

### Read from Azure Cosmos DB

```python
# Read from synapse link
productsDataFrame = spark.read.format("cosmos.olap")\
    .option("spark.synapse.linkedService", "cosmicworks_serv")\
    .option("spark.cosmos.container", "products")\
    .load()
```

### Write to Azure Cosmos DB


```python
# Write to OLTP
productsDataFrame.write.format("cosmos.oltp")\
    .option("spark.synapse.linkedService", "cosmicworks_serv")\
    .option("spark.cosmos.container", "products")\
    .mode('append')\
    .save()

# Write stream
query = productsDataFrame\
    .writeStream\
    .format("cosmos.oltp")\
    .option("spark.synapse.linkedService", "cosmicworks_serv")\
    .option("spark.cosmos.container", "products")\
    .option("checkpointLocation", "/tmp/runIdentifier/")\
    .outputMode("append")\
    .start()

query.awaitTermination()
```